In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
# Chemin pour enregistrer les fichiers pickle
datadir = '/home/chardong/y_identification/Venv/save_pkl/Fudge_Factor/'

In [3]:
totald = pd.read_pickle(datadir+"Py8_yj_jj_mc16ade_pd122_train_w_skim_05.pkl")

In [4]:
columns = list(totald.columns)
columns

['y_Reta',
 'y_Rphi',
 'y_weta2',
 'y_fracs1',
 'y_weta1',
 'y_wtots1',
 'y_Rhad',
 'y_Rhad1',
 'y_Eratio',
 'y_deltae',
 'y_noFF_Reta',
 'y_noFF_Rphi',
 'y_noFF_weta2',
 'y_noFF_fracs1',
 'y_noFF_weta1',
 'y_noFF_wtots1',
 'y_noFF_Rhad',
 'y_noFF_Rhad1',
 'y_noFF_Eratio',
 'y_noFF_deltae',
 'y_convRadius',
 'y_convType',
 'y_pt',
 'y_eta',
 'y_phi',
 'evt_mu',
 'y_jmin_dr',
 'y_IsTight',
 'y_IsLoose',
 'y_truth_pt',
 'y_truth_eta',
 'weight',
 'truth_label']

In [5]:
# Simple fonction de calcul des bornes basses et hautes pour les fudge factors
def FF(X, X_noFF):
    FFplus = X + (X_noFF - X) / 2
    FFminus = X - (X_noFF - X) / 2
    return FFplus, FFminus

# Fonction on shower-shape variables depending on their Fudge Factor to create the upper and bottom boundary.
Be carefull about X > X_noFF or X < X_noFF to correctly applied the function

TEST

In [6]:
plusy_Reta, minusy_Reta = FF(totald['y_Reta'], totald['y_noFF_Reta'])
plusy_Rphi, minusy_Rphi = FF(totald['y_Rphi'], totald['y_noFF_Rphi'])
print(plusy_Reta.head(1))
print(minusy_Reta.head(1))
print(plusy_Rphi.head(1))
print(minusy_Rphi.head(1))

33407159    0.950085
dtype: float32
33407159    0.946143
dtype: float32
33407159    0.969418
dtype: float32
33407159    0.974662
dtype: float32


In [7]:
# Liste des colonnes shower shape
shower_shape_columns = ['y_Reta', 'y_Rphi', 'y_weta2', 'y_fracs1', 'y_weta1', 'y_wtots1', 'y_Rhad', 'y_Rhad1', 'y_Eratio', 'y_deltae']

# Appliquer la fonction à chaque paire de colonnes
for col in shower_shape_columns:
    noFF_col = 'y_noFF_' + col.split('_')[1]
    if noFF_col in totald.columns:
        totald[f'{col}_FFplus'], totald[f'{col}_FFminus'] = FF(totald[col], totald[noFF_col])

#print(totald.head(5))

In [8]:
# Créer et sauvegarder le DataFrame avec FFminus
totald_FFminus = totald.copy()
for col in shower_shape_columns:
    totald_FFminus[col] = totald_FFminus[f'{col}_FFminus']
totald_FFminus.drop(columns=[f'{col}_FFplus' for col in shower_shape_columns] + [f'y_noFF_{col.split("_")[1]}' for col in shower_shape_columns], inplace=True)

with open(datadir + 'totald_FFminus_FF.pkl', 'wb') as f:
    pickle.dump(totald_FFminus, f)
print("DataFrame avec FFminus sauvegardé.")

# Créer et sauvegarder le DataFrame avec FFplus
totald_FFplus = totald.copy()
for col in shower_shape_columns:
    totald_FFplus[col] = totald_FFplus[f'{col}_FFplus']
totald_FFplus.drop(columns=[f'{col}_FFminus' for col in shower_shape_columns] + [f'y_noFF_{col.split("_")[1]}' for col in shower_shape_columns], inplace=True)

with open(datadir + 'totald_FFplus_FF.pkl', 'wb') as f:
    pickle.dump(totald_FFplus, f)
print("DataFrame avec FFplus sauvegardé.")

# Créer et sauvegarder le DataFrame avec noFF
totald_noFF = totald.copy()
for col in shower_shape_columns:
    noFF_col = 'y_noFF_' + col.split('_')[1]
    totald_noFF[col] = totald_noFF[noFF_col]
totald_noFF.drop(columns=[f'{col}_FFplus' for col in shower_shape_columns] + [f'{col}_FFminus' for col in shower_shape_columns], inplace=True)

with open(datadir + 'totald_noFF_FF.pkl', 'wb') as f:
    pickle.dump(totald_noFF, f)
print("DataFrame avec noFF sauvegardé.")

DataFrame avec FFminus sauvegardé.
DataFrame avec FFplus sauvegardé.
DataFrame avec noFF sauvegardé.


In [9]:
# Liste des colonnes shower shape de base
shower_shape_columns_base = [
    'y_Reta', 'y_Rphi', 'y_weta2', 'y_fracs1', 'y_weta1',
    'y_wtots1', 'y_Rhad', 'y_Rhad1', 'y_Eratio', 'y_deltae'
]

# Supprimer les colonnes des DataFrames
totald_FFminus.drop(columns=shower_shape_columns_base, inplace=True, errors='ignore')
totald_FFplus.drop(columns=shower_shape_columns_base, inplace=True, errors='ignore')
totald_noFF.drop(columns=shower_shape_columns_base, inplace=True, errors='ignore')

# Renommer les colonnes des DataFrames
for col in shower_shape_columns_base:
    totald_FFminus.rename(columns={f'{col}_FFminus': col}, inplace=True)
    totald_FFplus.rename(columns={f'{col}_FFplus': col}, inplace=True)
    totald_noFF.rename(columns={f'y_noFF_{col.split("_")[1]}': col}, inplace=True)

# Sauvegarder les DataFrames modifiés
totald_FFminus.to_pickle(datadir + 'totald_FFminus_FF.pkl')
totald_FFplus.to_pickle(datadir + 'totald_FFplus_FF.pkl')
totald_noFF.to_pickle(datadir + 'totald_noFF_FF.pkl')